In [16]:
import numpy as np
import glob
import pandas as pd
import re
import os
import json
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics as mt
from sklearn.metrics import matthews_corrcoef
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)
os.chdir("/Users/Lenovo/") #set your directory

In [19]:
DIR = "dataset/"
COLUMN_NAMES = ["isGenuine", "statuses_count", "followers_count", "friends_count", "favourites_count",
                "listed_count", "default_profile", "geo_enabled", "profile_use_background_image", "verified", "protected","id"]
COLUMN_NAMES_new = ["isGenuine", "statuses_count", "followers_count", "friends_count", "favourites_count",
                "listed_count", "verified","id"]
rus_users = pd.read_csv('dataset/rus_users.csv',na_filter=False)
account_files = glob.glob(DIR + 'genuine_accounts_users.csv')
account_files.append(DIR + "fake_followers_users.csv")
account_files.append(DIR + "social_spambots_1_users_.csv")
account_files.append(DIR + "social_spambots_2_users.csv")
account_files.append(DIR + "social_spambots_3_users.csv")
account_files.append(DIR + "traditional_spambots_1_users.csv")
account_files.append(DIR + "traditional_spambots_2_users.csv")
account_files.append(DIR + "traditional_spambots_3_users.csv")
account_files.append(DIR + "traditional_spambots_4_users.csv")
account_files.append(DIR + "rus_users.csv")
data = []
file_paths = []
for file_path in account_files:
    account_set = pd.read_csv(file_path, delimiter=',', dtype=str, error_bad_lines=False)
    account_set["isGenuine"] = 0
    if file_path == "dataset\\genuine_accounts_users.csv":
        account_set["isGenuine"] = 1
    data.append(account_set)
    file_paths.append(file_path)
file_paths

['dataset/genuine_accounts_users.csv',
 'dataset/fake_followers_users.csv',
 'dataset/social_spambots_1_users_.csv',
 'dataset/social_spambots_2_users.csv',
 'dataset/social_spambots_3_users.csv',
 'dataset/traditional_spambots_1_users.csv',
 'dataset/traditional_spambots_2_users.csv',
 'dataset/traditional_spambots_3_users.csv',
 'dataset/traditional_spambots_4_users.csv',
 'dataset/rus_users.csv']

In [20]:
############################## for Experiment-1 use 1083 accounts for genuine accounts dataset #################################
from numpy import loadtxt
ids = loadtxt("genuine_account_ids.txt", delimiter="\n", unpack=False)
################################################################################################################################

In [21]:
#use these for Experiment 1-2

ff_y = data[0]['isGenuine']
ga_y = data[1]['isGenuine']
ss1_y = data[2]['isGenuine']
ss2_y = data[3]['isGenuine']
ss3_y = data[4]['isGenuine']
ts1_y = data[5]['isGenuine']
ts2_y = data[6]['isGenuine']
ts3_y = data[7]['isGenuine']
ts4_y = data[8]['isGenuine']
ru_y = data[9]['isGenuine']

ff = data[0][COLUMN_NAMES].fillna(value=0)
#data[1]['id'] = data[1]['id'].astype(float) #use for Experiment-1 -> 1083 account, commented out for experiment 2-3
#data[1] = data[1][data[1]['id'].isin(ids)] #use for Experiment-1 -> 1083 account, commented out for experiment 2-3
ga = data[1][COLUMN_NAMES].fillna(value=0)
ss1 = data[2][COLUMN_NAMES].fillna(value=0)
ss2 = data[3][COLUMN_NAMES].fillna(value=0)
ss3 = data[4][COLUMN_NAMES].fillna(value=0)
ts1 = data[5][COLUMN_NAMES].fillna(value=0)
ts2 = data[6][COLUMN_NAMES].fillna(value=0)
ts3 = data[7][COLUMN_NAMES].fillna(value=0)
ts4 = data[8][COLUMN_NAMES].fillna(value=0)

ru = data[9][COLUMN_NAMES_new].fillna(value=0)
ru['default_profile'] = 1
ru['geo_enabled'] = 0
ru['profile_use_background_image'] = 0
ru['protected'] = 0
ru['verified'] = ru['verified'].map({'true': int(1), 'false': int(0)})
ru['id'] = ru['id'].map(lambda x: str(x))
ru = ru.fillna(value=0)
data[9] = ru

#print(ru)
ff = ff.drop('isGenuine', axis=1)
ga = ga.drop('isGenuine', axis=1)
ss1 = ss1.drop('isGenuine', axis=1)
ss2 = ss2.drop('isGenuine', axis=1)
ss3 = ss3.drop('isGenuine', axis=1)
ts1 = ts1.drop('isGenuine', axis=1)
ts2 = ts2.drop('isGenuine', axis=1)
ts3 = ts3.drop('isGenuine', axis=1)
ts4 = ts4.drop('isGenuine', axis=1)
ru = ru.drop('isGenuine', axis=1)
dataset_ = pd.DataFrame()
for i in range(len(data[:9])):
    #if i!=9 and i!=8 and i!=7 and i!=6 and i!=5 and i!=3 and i!=0: #extract the datasets that do not want to use for the experiment
    dataset_ = pd.concat([dataset_,data[i][COLUMN_NAMES]],sort=False)
dataset_['id'] = dataset_['id'].astype(float)
dataset = dataset_.sort_values(by='id')
y = dataset["isGenuine"]
X = dataset.drop('isGenuine', axis=1)
X = X.sort_values(by=['id'])
y = y.fillna(value= 0) #for 1 russian bot
#X = X.drop('id', axis=1)
X = X.fillna(value= 0)

In [52]:
#use this for experiment 3

os.chdir("/Users/Lenovo/dataset/")
#use the followings for scenarios that datasets are concatenated 
#train_dataset1 = pd.read_csv("vendorPurchased.csv", na_filter=False, lineterminator='\n', sep=',')
#train_dataset2 = pd.read_csv("verified.csv", na_filter=False, lineterminator='\n', sep=',')
#train_dataset3 = pd.read_csv("politicalBots.csv", na_filter=False, lineterminator='\n', sep=',')
#train_dataset = pd.concat([train_dataset1,train_dataset2,train_dataset3])
train_dataset = pd.read_csv("cresciRtbust.csv", na_filter=False, lineterminator='\n', sep=',')
train_score = train_dataset[['id','name','screen_name','statuses_count','followers_count','friends_count','favourites_count','listed_count','url','lang','time_zone','location','default_profile','default_profile_image','geo_enabled','profile_image_url','profile_banner_url','profile_use_background_image','profile_background_image_url_https','profile_text_color','profile_image_url_https','profile_sidebar_border_color','profile_background_tile','profile_sidebar_fill_color','profile_background_image_url','profile_background_color','profile_link_color','utc_offset','is_translator','follow_request_sent','protected','verified','notifications','description','contributors_enabled','following','created_at']]
train_score_y = train_dataset.iloc[:,42:]
train_score_y.columns = train_score_y.columns.str.strip() 
train_score_y = train_score_y['knownBot'].map({1:0,0:1})

In [53]:
#use this for experiment 3

train_score = train_score[["statuses_count", "followers_count", "friends_count", "favourites_count","listed_count", "default_profile", "geo_enabled", "profile_use_background_image", "verified", "protected","id"]]
train_score = train_score.fillna(value=0)
train_score['default_profile'] = train_score['default_profile'].map({True: 1, False: 0})
train_score['geo_enabled'] = train_score['geo_enabled'].map({True: 1, False: 0})
train_score['profile_use_background_image'] = train_score['profile_use_background_image'].map({True: 1, False: 0})
train_score['verified'] = train_score['verified'].map({True: 1, False: 0})
train_score['protected'] = train_score['protected'].map({True: 1, False: 0})

In [54]:
X = train_score
y = train_score_y

# EXPERIMENT 1

In [157]:
X = X.astype(float)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [130]:
sme = SMOTEENN(smote=SMOTE(k_neighbors=10))
train_score, train_score_y = sme.fit_resample(X, y)

gbc = GradientBoostingClassifier(loss='exponential')
abc = AdaBoostClassifier(n_estimators=1000, learning_rate=0.5, base_estimator=gbc)

model = abc.fit(train_score, train_score_y)

In [131]:
y_pred = model.predict(X_test)

In [132]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision Score", precision_score(y_test, y_pred))
print("Recall Score", recall_score(y_test, y_pred))
print("F1 Score", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))
print("MCC:", matthews_corrcoef(y_test, y_pred))
conf = mt.confusion_matrix(y_test,y_pred)
conf

Accuracy: 0.9880184331797235
Precision Score 0.8966942148760331
Recall Score 0.9954128440366973
F1 Score 0.9434782608695652
AUC: 0.9913027334937585
MCC: 0.9383959652592238


array([[1927,   25],
       [   1,  217]], dtype=int64)

# EXPERIMENT 2-3

In [55]:
os.chdir("/Users/Lenovo/dataset/")
#use the followings for scenarios that datasets are concatenated 
#test_dataset1 = pd.read_csv("vendorPurchased.csv", na_filter=False, lineterminator='\n', sep=',')
#test_dataset2 = pd.read_csv("verified.csv", na_filter=False, lineterminator='\n', sep=',')
#test_dataset3 = pd.read_csv("politicalBots.csv", na_filter=False, lineterminator='\n', sep=',')
#test_dataset = pd.concat([test_dataset1,test_dataset2,test_dataset3])
test_dataset = pd.read_csv("gilani.csv", na_filter=False, lineterminator='\n', sep=',')
test_score = test_dataset[['name','screen_name','statuses_count','followers_count','friends_count','favourites_count','listed_count','url','lang','time_zone','location','default_profile','default_profile_image','geo_enabled','profile_image_url','profile_banner_url','profile_use_background_image','profile_background_image_url_https','profile_text_color','profile_image_url_https','profile_sidebar_border_color','profile_background_tile','profile_sidebar_fill_color','profile_background_image_url','profile_background_color','profile_link_color','utc_offset','is_translator','follow_request_sent','protected','verified','notifications','description','contributors_enabled','following','created_at']]
test_score_y = test_dataset.iloc[:,42:]
test_score_y.columns = test_score_y.columns.str.strip() 
test_score_y = test_score_y['knownBot'].map({1:0,0:1})

In [56]:
test_score = test_score[["statuses_count", "followers_count", "friends_count", "favourites_count","listed_count", "default_profile", "geo_enabled", "profile_use_background_image", "verified", "protected"]]
test_score = test_score.fillna(value=0)
test_score['default_profile'] = test_score['default_profile'].map({True: 1, False: 0})
test_score['geo_enabled'] = test_score['geo_enabled'].map({True: 1, False: 0})
test_score['profile_use_background_image'] = test_score['profile_use_background_image'].map({True: 1, False: 0})
test_score['verified'] = test_score['verified'].map({True: 1, False: 0})
test_score['protected'] = test_score['protected'].map({True: 1, False: 0})

In [57]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train = X.drop(columns='id')
y_train = y
X_test = test_score
y_test = test_score_y

In [58]:
sme = SMOTEENN(smote=SMOTE(k_neighbors=10))
X_train, y_train = sme.fit_resample(X_train, y_train)

gbc = GradientBoostingClassifier(loss='exponential')
abc = AdaBoostClassifier(n_estimators=1000, learning_rate=0.5, base_estimator=gbc)

model = abc.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [59]:
# Accuracy: 0.988865692414753
# Precision Score 0.9744318181818182
# Recall Score 0.98
# F1 Score 0.9772079772079771
# ROC/AUC Score 0.9984616901038245

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

y_scores = model.decision_function(X_test)
print("Precision Score", precision_score(y_test, y_pred))
print("Recall Score", recall_score(y_test, y_pred))
print("F1 Score", f1_score(y_test, y_pred))
print("ROC/AUC Score", roc_auc_score(y_test, y_pred))

Accuracy: 0.4905356423681031
Precision Score 0.5318204242723237
Recall Score 0.7733142037302726
F1 Score 0.6302250803858521
ROC/AUC Score 0.4509362570533824
